# Diffie Hellman Key Exchange

This notebook is an edited extract of notebook **Part 6:  Ciphers and Key exchange in Python 3.x** 
by Martin Weissman. This Notebook appears on Martin Weissman's page $[1]$ detailed in the **Sources** 
section below. 

**Imported functions.** Run the cell below to import functions needed in this notebook. 

In [5]:
# RUN THIS CELL
from random import randint, SystemRandom 
from cryptography_lecture_functions import is_prime 
print("Imports completed")


Imports completed


Some hybrid systems may require that both parties possess the same common key for the encryption and decryption of the data/message.

If the security of communication rests on possession of a common key, then we're left with the following problem:  how do the two parties agree on a common key, especially if they are far apart and communicating over an insecure channel?  

A clever solution to this problem was published in 1976 by Whitfield Diffie and Martin Hellman, and so it's called Diffie-Hellman key exchange.  It takes advantage of modular arithmetic:  the existence of a primitive root (modulo a prime) and the difficulty of solving the **discrete logarithm** problem.  

This part complements Chapter 6 of [An Illustrated Theory of Numbers](http://illustratedtheoryofnumbers.com).

## Key exchange

Now we study Diffie-Hellman **key exchange**, a remarkable way for two parties to share a secret without ever needing to directly communicate the secret with each other.  Their method is based on properties of modular exponentiation and the existence of a **primitive root** modulo prime numbers.   

### Primitive roots and Sophie Germain primes

If $p$ is a prime number, and $\mathrm{gcd}(a,p) = 1$, then recall Fermat's Little Theorem:  $$a^{p-1} \equiv 1 \text{ mod } p.$$
It may be the case that $a^\ell \equiv 1$ mod $p$ for some smaller (positive) value of $\ell$ however.  The smallest such positive value of $\ell$ is called the **order** (multiplicative order, to be precise) of $a$ modulo $p$, and it is always a divisor of $p-1$.

The following code determines the order of a number, mod $p$, with a brute force approach.

In [8]:
def mult_order(a,p):
    '''
    Determines the (multiplicative) order of an integer
    a, modulo p.  Here p is prime, and GCD(a,p) = 1.
    If bad inputs are used, this might lead to a 
    never-ending loop!
    '''
    current_number = a % p
    current_exponent = 1
    while current_number != 1:
        current_number = (current_number * a) % p
        current_exponent = current_exponent + 1
    return current_exponent
    

In [9]:
for j in range(1,37):
    print("The multiplicative order of {} modulo 37 is {}".format(j,mult_order(j,37)))
    # These orders should all be divisors of 36.

The multiplicative order of 1 modulo 37 is 1
The multiplicative order of 2 modulo 37 is 36
The multiplicative order of 3 modulo 37 is 18
The multiplicative order of 4 modulo 37 is 18
The multiplicative order of 5 modulo 37 is 36
The multiplicative order of 6 modulo 37 is 4
The multiplicative order of 7 modulo 37 is 9
The multiplicative order of 8 modulo 37 is 12
The multiplicative order of 9 modulo 37 is 9
The multiplicative order of 10 modulo 37 is 3
The multiplicative order of 11 modulo 37 is 6
The multiplicative order of 12 modulo 37 is 9
The multiplicative order of 13 modulo 37 is 36
The multiplicative order of 14 modulo 37 is 12
The multiplicative order of 15 modulo 37 is 36
The multiplicative order of 16 modulo 37 is 9
The multiplicative order of 17 modulo 37 is 36
The multiplicative order of 18 modulo 37 is 36
The multiplicative order of 19 modulo 37 is 36
The multiplicative order of 20 modulo 37 is 36
The multiplicative order of 21 modulo 37 is 18
The multiplicative order of 22

A theorem of Gauss states that, if $p$ is prime, there exists an integer $b$ whose order is precisely $p-1$ (as big as possible!).  Such an integer is called a **primitive root** modulo $p$.  For example, the previous computation found 12 primitive roots modulo $37$:  they are 2,5,13,15,17,18,19,20,22,24,32,35.  To see these illustrated (mod 37), check out [this poster](https://fineartamerica.com/featured/epicycles-modulo-37-martin-weissman.html) (yes, that is blatant self-promotion!)

For everything that follows, suppose that $p$ is a prime number.  Not only do primitive roots *exist* mod $p$, but they are pretty common.  In fact, the number of primitive roots mod $p$ equals $\phi(p-1)$, where $\phi$ denotes Euler's totient.  On average, $\phi(n)$ is about $6 / \pi^2$ times $n$ (for positive integers $n$).  While numbers of the form $p-1$ are not "average", one still expects that $\phi(p-1)$ is a not-very-small fraction of $p-1$.  You should not have to look very far if you want to *find* a primitive root.

The more difficult part, in practice, is determining whether a number $b$ is or is not a primitive root modulo $p$.  When $p$ is very large (like hundreds or thousands of digits), $p-1$ is also very large.  It is certainly not practical to cycle all the powers (from $1$ to $p-1$) of $b$ to determine whether $b$ is a primitive root!

The better approach, sometimes, is to use the fact that the multiplicative order of $b$ must be a **divisor** of $p-1$.  If one can find all the divisors of $p-1$, then one can just check whether $b^d \equiv 1$ mod $p$ for each divisor $d$.  This makes the problem of determining whether $b$ is a primitive root just about as hard as the problem of factoring $p-1$.  This is a hard problem, in general!

But, for the application we're interested in, we will want to have a large prime number $p$ **and** a primitive root mod $p$.  The easiest way to do this is to use a **Sophie Germain** prime $q$.  A Sophie Germain prime is a prime number $q$ such that $2q + 1$ is also prime.  When $q$ is a Sophie Germain prime, the resulting prime $p = 2q + 1$ is called a **safe prime**.

Observe that when $p$ is a safe prime, the prime decomposition of $p-1$ is 
$$p-1 = 2 \cdot q.$$
That's it.  So the possible multiplicative orders of an element $b$, mod $p$, are the divisors of $2q$, which are
$$1, 2, q, \text{ or } 2q.$$
In order to check whether $b$ is a primitive root, modulo a safe prime $p = 2q + 1$, we must check just three things:  is $b \equiv 1$, is $b^2 \equiv 1$, or is $b^q \equiv 1$, mod $p$?  If the answer to these three questions is NO, then $b$ is a primitive root mod $p$.

In [12]:
def is_primroot_safe(b,p):
    '''
    Checks whether b is a primitive root modulo p,
    when p is a safe prime.  If p is not safe,
    the results will not be good!
    '''
    q = (p-1) // 2   # q is the Sophie Germain prime
    if b % p == 1:  # Is the multiplicative order 1?
        return False
    if (b*b) % p == 1:  # Is the multiplicative order 2?
        return False
    if pow(b,q,p) == 1:  # Is the multiplicative order q?
        return False
    return True  # If not, then b is a primitive root mod p.

This would not be very useful if we couldn't find Sophie Germain primes.  Fortunately, they are not so rare.  The first few are 2, 3, 5, 11, 23, 29, 41, 53, 83, 89.  It is expected, but unproven that there are infinitely many Sophie Germain primes.  In practice, they occur fairly often.  If we consider numbers of magnitude $N$, about $1 / \log(N)$ of them are prime.  Among such primes, we expect about $1.3 / \log(N)$ to be Sophie Germain primes.  In this way, we can expect to stumble upon Sophie Germain primes if we search for a bit (and if $\log(N)^2$ is not too large).

The code below tests whether a number $p$ is a Sophie Germain prime.  We construct it by simply testing whether $p$ and $2p+1$ are both prime.  We use the function `is_prime` function imported from the module `cryptography_lecture_function` (and originally based on Martin Weissman's code) in order to test whether each is prime.

In [13]:
def is_SGprime(p):
    '''
    Tests whether p is a Sophie Germain prime
    '''
    if is_prime(p):  # A bit faster to check whether p is prime first.
        if is_prime(2*p + 1):  # and *then* check whether 2p+1 is prime.
            return True
    return False

Let's test this out by finding the Sophie Germain primes up to 100, and their associated safe primes.

In [14]:
for j in range(1,100):
    if is_SGprime(j):
        print(j, 2*j+1)

2 5
3 7
5 11
11 23
23 47
29 59
41 83
53 107
83 167
89 179


Next, we find the first 100-digit Sophie Germain prime!  This might take a minute!

In [15]:
test_number = 10**99 # Start looking at the first 100-digit number, which is 10^99.
while not is_SGprime(test_number):
    test_number = test_number + 1
print(test_number)

1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000044239


In the seconds or minutes your computer was running, it checked the primality of almost 90 thousand numbers, each with 100 digits.  Not bad!

### The Diffie-Hellman protocol

When we study protocols for secure communication, we must keep track of the communicating parties (often called Alice and Bob), and **who** has knowledge of **what** information.  We assume at all times that the "wire" between Alice and Bob is tapped -- anything they say to each other is actively monitored, and is therefore **public** knowledge.  We also assume that what happens on Alice's private computer is private to Alice, and what happens on Bob's private computer is private to Bob.  Of course, these last two assumptions are big assumptions -- they point towards the danger of computer viruses which infect computers and can violate such privacy!

The goal of the Diffie-Hellman protocol is -- at the end of the process -- for Alice and Bob to **share a secret** without ever having communicated the secret with each other.  The process involves a series of modular arithmetic calculations performed on each of Alice and Bob's computers.

The process begins when Alice or Bob creates and publicizes a **large prime number** `p` and a **primitive root** `g` modulo `p`.  It is best, for efficiency and security, to choose a **safe** prime `p`.  Alice and Bob can create their own safe prime, or choose one from a public list online, e.g., from the [RFC 3526 memo](https://tools.ietf.org/html/rfc3526).  Nowadays, it's common to take `p` with 2048 bits, i.e., a prime which is between $2^{2046}$ and $2^{2047}$ (a number with 617 decimal digits!).

For the purposes of this introduction, we use a smaller safe prime, with about 256 bits.  We use the `SystemRandom` function (imported above) of the `random` package to create a good random prime.  It is not so much of an issue here, but in general one must be very careful in cryptography that one's "random" numbers are really "random"!  The `SystemRandom` function uses chaotic properties of your computer's innards in order to initialize a random number generator, and is considered cryptographically secure.

In [16]:
r = SystemRandom().getrandbits(256)
print("The random integer is {}".format(r))
print("with binary expansion {}".format(bin(r)))  #  r is an integer constructed from 256 random bits.
print("with bit-length {}.".format(len(bin(r)) - 2))  # In case you want to check.  Remember '0b' is at the beginning.

The random integer is 107587964890221801931860012533034522593833148083283411480874946891955582623568
with binary expansion 0b1110110111011100101000010101001100011011100001110001111010000010011010010000110010001000110000111011100001111011000110111001101111000010100101011001000100010010001110100001111001011100000011100010111010010011010000001011001100010111100101010101111101010000
with bit-length 256.


In [17]:
def getrandSGprime(bitlength):
    '''
    Creates a random Sophie Germain prime p with about 
    bitlength bits.
    '''
    while True:
        p = SystemRandom().getrandbits(bitlength) # Choose a really random number.
        if is_SGprime(p):
            return p    

The function above searches and searches among random numbers until it finds a Sophie Germain prime.  The (possibly endless!) search is performed with a `while True:` loop that may look strange.  The idea is to stay in the loop until such a prime is found.  Then the `return p` command returns the found prime as output and halts the loop.  One must be careful with `while True` loops, since they are structured to run forever -- if there's not a loop-breaking command like `return` or `break` inside the loop, your computer will be spinning for a long time.

In [18]:
q = getrandSGprime(256) # A random ~256 bit Sophie Germain prime
p = 2*q + 1 # And its associated safe prime

print("p is ",p)  # Just to see what we're working with.
print("q is ",q)

p is  200682515542174909530148138931236775568423564825161225412936682163636812288523
q is  100341257771087454765074069465618387784211782412580612706468341081818406144261


Next we find a primitive root, modulo the safe prime `p`.

In [19]:
def findprimroot_safe(p):
    '''
    Finds a primitive root, 
    modulo a safe prime p.
    '''
    b = 2  # Start trying with 2.
    while True:  # We just keep on looking.
        if is_primroot_safe(b,p):
            return b
        b = b + 1 # Try the next base.  Shouldn't take too long to find one!

In [20]:
g = findprimroot_safe(p)
print(g)

2


The pair of numbers $(g, p)$, the primitive root and the safe prime, chosen by either Alice or Bob, is now made public.  They can post their $g$ and $p$ on a public website or shout it in the streets.  It doesn't matter.  They are just tools for their secret-creation algorithm below.

### Alice and Bob's private secrets

Next, Alice and Bob invent **private** secret numbers $a$ and $b$.  They do not tell *anyone* these numbers.  Not each other.  Not their family.  Nobody.  They don't write them on a chalkboard, or leave them on a thumbdrive that they lose.  These are really secret.

But they don't use their phone numbers, or social security numbers.  It's best for Alice and Bob to use a secure random number generator on their separate private computers to create $a$ and $b$.  They are often 256 bit numbers in practice, so that's what we use below.

In [ ]:
a = SystemRandom().getrandbits(256)  # Alice's secret number
b = SystemRandom().getrandbits(256)  # Bob's secret number

print("Only Alice should know that a = {}".format(a))
print("Only Bob should know that b = {}".format(b))

print("But everyone can know p = {} and g = {}".format(p,g))

Now Alice and Bob use their secrets to generate new numbers.  Alice computes the number 
$$A = g^a \text{ mod } p,$$
and Bob computes the number
$$B = g^b \text{ mod } p.$$


In [ ]:
A = pow(g,a,p)  # This would be computed on Alice's computer.
B = pow(g,b,p)  # This would be computed on Bob's computer.

Now Alice and Bob do something that seems very strange at first.  Alice sends Bob her new number $A$ and Bob sends Alice his new number $B$.  Since they are far apart, and the channel is insecure, we can assume everyone in the world now knows $A$ and $B$.

In [ ]:
print("Everyone knows A = {} and B = {}.".format(A,B))

Now Alice, on her private computer, computes $B^a$ mod $p$.  She can do that because everyone knows $B$ and $p$, and she knows $a$ too.

Similarly, Bob, on his private computer, computes $A^b$ mod $p$.  He can do that because everyone knows $A$ and $p$, and he knows $b$ too.

Alice and Bob **do not** share the results of their computations!

In [ ]:
print(pow(B,a,p))  # This is what Alice computes.

In [ ]:
print(pow(A,b,p))  # This is what Bob computes.

Woah!  What happened?  In terms of exponents, it's elementary.  For
$$B^a = (g^{b})^a = g^{ba} = g^{ab} = (g^a)^b = A^b.$$
So these two computations yield the same result (mod $p$, the whole way through).

In the end, we find that Alice and Bob **share a secret**.  We call this secret number $S$.
$$S = B^a = A^b.$$

In [ ]:
S = pow(B,a,p)  # Or we could have used pow(A,b,p)
print(S)

This common secret $S$ can be used as a key for Alice and Bob to communicate hereafter.  For example, they might use $S$ (converted to a string, if needed) as the key for a Vigenère cipher, and chat with each other knowing that only they have the secret key to encrypt and decrypt their messages.


#### Simple Example.

Let's suppose that the data encryption/decryption system being used by Bob and Alice is (like in the core sections of the project) the Vigenère cipher with encryption/decryption functions `vigenere_encrypt` and `vigenere_decrypt`. 

Let's also suppose that Alice has the following secret message that she wants to send to Bob. 

```python
plaintext_message  = '''Did you hear that the American Mathematical Society \ 
                        has an annual textbook sale? \
                        It's 40 percent off for members and 25 percent off for everyone else.'''
``` 
Then Alice uses the common secret $S$ (converted to a string) to encrypt `plaintext_message`. 

```python 
cipher_text = vigenere_encrypt(plaintext_message, str(S))
``` 

Alice sends `cipher_text` to Bob (via an insecure channel). When Bob receives the message he is 
able to decrypt it since he is also in possession of the key $S$. 

```python
transmitted_message = vignenere_decrypt(cipher_text, str(S))
``` 

And so Bob will be able to take advantage of the AMS book sale! 


### Is the Diffie Hellman protocol secure?

To have confidence in this protocol, one needs to be convinced that their secret is truly secret!  The public has a lot of information:  they know 
1.  the prime $p$, 
2.  the primitive root $g$, 
3.  the number $A = g^a$ (mod $p$), and 
4.  the number $B = g^b$ (mod $p$).  

If the public could **figure out** either $a$ or $b$, they could figure out the secret (by raising $A^b$ or $B^a$ like Alice and Bob did).  

This is the essence of the **discrete logarithm problem**.  If we know the value of $g^a$ mod $p$, can we figure out the possible value(s) of $a$?  If this were ordinary arithmetic, we would say that $a = \log_g(A)$.  But this is modular arithmetic, and there's no easy way to figure out such logarithms.  The values of $g^a$ tend to bounce all over the place, mod $p$, especially since we chose $a$ to be pretty large (256 bits!).  

The security of the Diffie-Hellman protocol, i.e., the security of Alice and Bob's shared secret, depends on the *difficulty* of the discrete logarithm problem.  When $p$ is a large (e.g. 2048 bits) safe prime, and $a$ and $b$ are suitably large (roughly 256 bits), there seems to be no way to solve the discrete logarithm problem mod $p$ in any reasonable amount of time.  Someday we might have quantum computers to quickly solve discrete logarithm problems, and the Diffie-Hellman protocol will not be secure.  But for now, Alice and Bob's secret key seems safe.

### Exercises

1.  How many Sophie Germain primes are there between 1 and 1000000?  What proportion of primes in this range are Sophie Germain primes?

2.  [It is expected](https://en.wikipedia.org/wiki/Artin%27s_conjecture_on_primitive_roots) that there are infinitely primes $p$ such that 2 is a primitive root mod $p$.  Study the density of such primes.  For example, among the primes up to 1000, how often is $2$ a primitive root?  Does this density seem to change?  

3.  Adapt Diffie-Hellman to work with a group of *three* parties who wish to share a common secret.  Hint:  the common secret will have the form $g^{abc}$, and other exponents like $g^a$, $g^b$, $g^c$, $g^{ab}$, $g^{bc}$, $g^{ac}$ will be public information.

4.  Sadly, Alice and Bob have agreed to use the primitive root $g = 3$ and the prime $p = 65537$.  Listening into their conversation, you intercept the following: $A = 40360$ and $B = 21002$ and the ciphertext is `$;6HWD;P5LVJ99W+EH9JVx=I:V7ESpGC^`.  If you know that they use a protocol with a Vigenère cipher, with key equal the string associated to their secret $S$, what is the plaintext message?  Hint:  you should be able to solve the discrete logarithm problem with a brute force attack.



### Sources. 

The above material is an edited extract from  the Jupyter notebook **Part 6:  Ciphers and Key exchange in Python 3.x** that can be found on the following page. 

[1] Martin H. Weissman. [Python for number Theory (Jupyter Notebooks)](http://illustratedtheoryofnumbers.com/prog.html)